In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
import numpy as np
from Module.sam import sam_model_registry, SamPredictor
from Module.sam.utils.transforms import ResizeLongestSide

import onnx
import onnxsim
from onnxconverter_common.float16 import convert_float_to_float16_model_path
from onnxconverter_common import auto_mixed_precision

C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
C:\Users\danusys16\anaconda3\envs\sam\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
D:\015_segmentation\Custom_SAM\Module\sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with Module.sam.modeling.tiny_vit_sam.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model

## Encoder

In [4]:
model_type = 'vit_b'
checkpoint = "../runs/241115_SAM_ViT_b_dataset_V2_ft_v1/best.pth"

In [5]:
sam = sam_model_registry[model_type](checkpoint=checkpoint)
#sam.to(device)
sam.eval();

D:\015_segmentation\Custom_SAM\Module\sam\build_sam.py:154: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


In [6]:
class Model(torch.nn.Module):
    def __init__(self, image_size, checkpoint, model_type):
        super().__init__()
        self.sam = sam_model_registry[model_type](checkpoint=checkpoint)
        self.sam.to(device='cpu')
        self.predictor = SamPredictor(self.sam)
        self.image_size = image_size

    def forward(self, x):
        self.predictor.set_torch_image(x, (self.image_size))
        return self.predictor.get_image_embedding()


In [7]:
model =Model(image_size=(1024,1024),
             checkpoint=checkpoint,
             model_type=model_type)

model.to(device);
model.eval();

### Dummy Input

In [8]:
bs = 1
img_size = 1024

In [9]:
transform = ResizeLongestSide(sam.image_encoder.img_size)

In [10]:
pixel_mean = torch.Tensor([123.675, 116.28, 103.53]).view(-1, 1, 1)
pixel_std = torch.Tensor([58.395, 57.12, 57.375]).view(-1, 1, 1)

In [11]:
dummy_input = np.zeros((sam.image_encoder.img_size, sam.image_encoder.img_size, 3), dtype=np.uint8)
dummy_input = transform.apply_image(dummy_input)
dummy_input = torch.as_tensor(dummy_input, device='cpu')
dummy_input = dummy_input.permute(2, 0, 1).contiguous()[None, :, :, :]

In [12]:
dummy_input.shape

torch.Size([1, 3, 1024, 1024])

### ONNX Export

In [13]:
output_path = "./test.onnx"

In [14]:
with torch.no_grad():
    img = dummy_input.to(device)
    y = model(img) 

In [15]:
input_names=['input']
output_names=['output']
# opset 17 support layernorm for convnext


In [16]:
torch.onnx.export(model, 
                  img, 
                  output_path, 
                  verbose=False, opset_version=17, 
                  input_names=input_names,
                  output_names=output_names,          # the ONNX version to export the model to
                  # export_params=True,
                  do_constant_folding=True,
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                'output' : {0 : 'batch_size'}})

D:\015_segmentation\Custom_SAM\Module\sam\predictor.py:81: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  and transformed_image.shape[1] == 3
D:\015_segmentation\Custom_SAM\Module\sam\predictor.py:82: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  and max(*transformed_image.shape[2:]) == self.model.image_encoder.img_size
D:\015_segmentation\Custom_SAM\Module\sam\modeling\image_encoder.py:258: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be 

In [17]:
onnx_model = onnx.load(output_path)

In [20]:
simplified_onnx_model_path =  os.path.join('onnx', "test_simplified.onnx")


In [21]:
print(f'Generating {simplified_onnx_model_path} ...')
simplified_onnx_model, success = onnxsim.simplify(onnx_model)
onnx.save_model(simplified_onnx_model, simplified_onnx_model_path, save_as_external_data=True, all_tensors_to_one_file=False)
print('done')

Generating onnx\test_simplified.onnx ...
done


In [5]:
input_onnx_model = simplified_onnx_model_path
output_onnx_model = os.path.join('onnx_export', "model.fp16.simplified.onnx")

In [27]:
test_data = {"input": np.random.rand(1, 3,  sam.image_encoder.img_size,  sam.image_encoder.img_size).astype(np.float32)-0.5}

In [30]:
def validate(res1, res2):
    for r1, r2 in zip(res1, res2):
        if not np.allclose(r1, r2, rtol=1e-5, atol=1e-7):
            return False
    return True

In [6]:
model_fp16 = convert_float_to_float16_model_path(input_onnx_model, keep_io_types=True)

PermissionError: [Errno 13] Permission denied: 'D:\\015_segmentation\\Custom_SAM\\notebooks\\onnx\\tmpwz1z_a8b'